# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import re
import nltk
import numpy as np
from pandas import read_sql_table
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

In [32]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = read_sql_table('disaster_messages', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)
category_names = Y.columns

In [33]:
df.related.unique()

array([1, 0], dtype=int64)

In [34]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [35]:
def tokenize(text: str) -> list:
    """
    INPUT:
    text - string containing the message
    
    OUTPUT:
    tokens - List of tokens

    This function tokenizes the input text by using URL replacement, normalization, punctuation removal, tokenization,
    lemmatization and stemming
    """
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'   
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [36]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [37]:
# Splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [38]:
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001B36286E520>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [39]:
y_pred = pipeline.predict(X_test)

In [40]:
print(classification_report(Y_test, y_pred, target_names=category_names)) # output_dict=True

                        precision    recall  f1-score   support

               related       0.81      0.98      0.88      3959
               request       0.90      0.43      0.58       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.79      0.62      0.70      2156
          medical_help       0.71      0.03      0.05       431
      medical_products       0.85      0.09      0.16       264
     search_and_rescue       0.78      0.05      0.09       151
              security       0.00      0.00      0.00       106
              military       0.75      0.03      0.07       175
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.20      0.32       344
                  food       0.87      0.42      0.57       586
               shelter       0.88      0.25      0.38       487
              clothing       0.73      0.10      0.18        79
                 money       0.86      

C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# specify parameters for grid search
parameters = {
   'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
   'clf__n_estimators': [50, 100, 200],
   'clf__min_samples_split': [2, 3, 4]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.